In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [18]:
housing_data = pd.read_csv("AmesHousing.tsv",delimiter = "\t")

In [19]:
def transform_features(data):
    # column names with count of null values
    isnull_count = data.isnull().sum()
    # number of rows in data
    row_count =  len(data)
    # Five percentage of total number of rows
    row_count_fiveperc = len(data)/20
    
    # columns that can be dropped which have more than 5 percentage null values
    isnull_count_filtered = isnull_count[isnull_count > row_count_fiveperc].sort_values()
    columns_drop = isnull_count_filtered.index
    # Data after dropping columns which have more than 5 percentage null values
    data = data.drop(columns_drop,axis=1)
    
    # selecting text columns
    text_data = data.select_dtypes(include=['object'])
    # derive count of null values in columns and sort on descending order
    isnull_count = text_data.isnull().sum().sort_values(ascending = False)
    # derive columns that have 1 or more than 1 null values
    columns_drop = isnull_count[isnull_count > 0].index
    # get data after dropping columns
    data = data.drop(columns_drop,axis=1)
    
    # selecting numerical data which has null values less than 5 percentage
    numeric_columns = data.select_dtypes(include=['integer','float'])
    null_numeric_count = numeric_columns.isnull().sum() 
    # selecting numerical columns which has null values less than 5 percentage
    numeric_null_data = null_numeric_count[(null_numeric_count > 0) & (null_numeric_count < len(data)/20)]
   
    # get most common value of each above selected columns
    common_dictionary = data[numeric_null_data.index].mode().to_dict(orient='records')[0]
    
    # to fill up missing values with most common values
    data = data.fillna(common_dictionary)
    
    # to create new features 
    data["years_before_sale"] = data["Yr Sold"] - data["Year Built"]
    data['Years Since Remod'] = data['Yr Sold'] - data['Year Remod/Add']
    
    # to drop some features
    data = data.drop([1702, 2180, 2181],axis=0)
    data = data.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return data

In [20]:
def train_test(data):
    train = data[0:1460]
    test  = data[1460:]
    #Include only numeric and floats
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(numeric_train[features],numeric_train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"],predictions)
    rmse = np.sqrt(mse)
    return rmse

In [21]:
data = transform_features(housing_data)
data_features = select_features(data)
rmse = train_test(data_features)
print(rmse)

33367.2871834


In [22]:
print(data.dtypes)

MS SubClass            int64
MS Zoning             object
Lot Area               int64
Street                object
Lot Shape             object
Land Contour          object
Utilities             object
Lot Config            object
Land Slope            object
Neighborhood          object
Condition 1           object
Condition 2           object
Bldg Type             object
House Style           object
Overall Qual           int64
Overall Cond           int64
Roof Style            object
Roof Matl             object
Exterior 1st          object
Exterior 2nd          object
Mas Vnr Area         float64
Exter Qual            object
Exter Cond            object
Foundation            object
BsmtFin SF 1         float64
BsmtFin SF 2         float64
Bsmt Unf SF          float64
Total Bsmt SF        float64
Heating               object
Heating QC            object
Central Air           object
1st Flr SF             int64
2nd Flr SF             int64
Low Qual Fin SF        int64
Gr Liv Area   

In [23]:
# select numeric columns from transformed data
data_numeric = data.select_dtypes(include=['integer','float'])
print(data_numeric)

      MS SubClass  Lot Area  Overall Qual  Overall Cond  Mas Vnr Area  \
0              20     31770             6             5         112.0   
1              20     11622             5             6           0.0   
2              20     14267             6             6         108.0   
3              20     11160             7             5           0.0   
4              60     13830             5             5           0.0   
5              60      9978             6             6          20.0   
6             120      4920             8             5           0.0   
7             120      5005             8             5           0.0   
8             120      5389             8             5           0.0   
9              60      7500             7             5           0.0   
10             60     10000             6             5           0.0   
11             20      7980             6             7           0.0   
12             60      8402             6          

[2927 rows x 34 columns]


In [24]:
# derive correlation of above numeric fields with SalePrice
data_corr = data_numeric.corr()

In [25]:
data_corr_salePrice = data_corr["SalePrice"].abs().sort_values()

In [26]:
print(data_corr_salePrice)

BsmtFin SF 2         0.006127
Misc Val             0.019273
Yr Sold              0.030358
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
years_before_sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

In [27]:
# lets keep only those columns with correlation co-efficient >= 0.4 and drop other columns 
columns_drop = data_corr_salePrice[data_corr_salePrice < 0.4].index
data_transformed = data.drop(columns_drop,axis=1)

In [28]:
print(data_transformed)

     MS Zoning Street Lot Shape Land Contour Utilities Lot Config Land Slope  \
0           RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
1           RH   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2           RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
3           RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
4           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
5           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
6           RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
7           RL   Pave       IR1          HLS    AllPub     Inside        Gtl   
8           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
9           RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
10          RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
11          RL   Pave       IR1         

In [29]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [30]:
# Lets see if we still carry above nominal features then store them in transform_category_columns
transform_catergory_columns = []
for col in nominal_features:
    if col in data_transformed.columns:
        transform_catergory_columns.append(col)      

In [31]:
## How many unique values in each categorical column?
data_uniqueness = data_transformed[transform_catergory_columns].apply(lambda col: len(col.value_counts())).sort_values()
#If count of uniqueness is greater than 10 then drop those columns
drop_columns = data_uniqueness[data_uniqueness > 10].index
data_transformed = data_transformed.drop(drop_columns,axis=1)
print(data_transformed)

     MS Zoning Street Lot Shape Land Contour Utilities Lot Config Land Slope  \
0           RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
1           RH   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2           RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
3           RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
4           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
5           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
6           RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
7           RL   Pave       IR1          HLS    AllPub     Inside        Gtl   
8           RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   
9           RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
10          RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
11          RL   Pave       IR1         

In [32]:
#select text columns and convert them to categorical
text_cols = data_transformed.select_dtypes(include=["object"])
for col in text_cols:
    data_transformed[col]= data_transformed[col].astype("category")
# get dummy columns
dummy_columns = pd.get_dummies(data_transformed.select_dtypes(include=["category"]))
# append dummy columns
data_transformed = pd.concat([data_transformed,dummy_columns],axis=1)


In [33]:
# now two functions are ready.
def transform_features(data):
    # column names with count of null values
    isnull_count = data.isnull().sum()
    # number of rows in data
    row_count =  len(data)
    # Five percentage of total number of rows
    row_count_fiveperc = len(data)/20
    
    # columns that can be dropped which have more than 5 percentage null values
    isnull_count_filtered = isnull_count[isnull_count > row_count_fiveperc].sort_values()
    columns_drop = isnull_count_filtered.index
    # Data after dropping columns which have more than 5 percentage null values
    data = data.drop(columns_drop,axis=1)
    
    # selecting text columns
    text_data = data.select_dtypes(include=['object'])
    # derive count of null values in columns and sort on descending order
    isnull_count = text_data.isnull().sum().sort_values(ascending = False)
    # derive columns that have 1 or more than 1 null values
    columns_drop = isnull_count[isnull_count > 0].index
    # get data after dropping columns
    data = data.drop(columns_drop,axis=1)
    
    # selecting numerical data which has null values less than 5 percentage
    numeric_columns = data.select_dtypes(include=['integer','float'])
    null_numeric_count = numeric_columns.isnull().sum() 
    # selecting numerical columns which has null values less than 5 percentage
    numeric_null_data = null_numeric_count[(null_numeric_count > 0) & (null_numeric_count < len(data)/20)]
   
    # get most common value of each above selected columns
    common_dictionary = data[numeric_null_data.index].mode().to_dict(orient='records')[0]
    
    # to fill up missing values with most common values
    data = data.fillna(common_dictionary)
    
    # to create new features 
    data["years_before_sale"] = data["Yr Sold"] - data["Year Built"]
    data['Years Since Remod'] = data['Yr Sold'] - data['Year Remod/Add']
    
    # to drop some features
    data = data.drop([1702, 2180, 2181],axis=0)
    data = data.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return data

def select_features(data, coeff_threshold=0.4, uniq_threshold=10):
    data_numeric = data.select_dtypes(include=['integer','float'])
    # derive correlation of above numeric fields with SalePrice
    data_corr = data_numeric.corr()
    data_corr_salePrice = data_corr["SalePrice"].abs().sort_values()
    # lets keep only those columns with correlation co-efficient >= 0.4 and drop other columns 
    columns_drop = data_corr_salePrice[data_corr_salePrice < coeff_threshold].index
    data_transformed = data.drop(columns_drop,axis=1)
    
    ## Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    # Lets see if we still carry above nominal features then store them in transform_category_columns
    transform_catergory_columns = []
    for col in nominal_features:
        if col in data_transformed.columns:
            transform_catergory_columns.append(col)       
    ## How many unique values in each categorical column?
    data_uniqueness = data_transformed[transform_catergory_columns].apply(lambda col: len(col.value_counts())).sort_values()
    #If count of uniqueness is greater than 10 then drop those columns
    drop_columns = data_uniqueness[data_uniqueness > uniq_threshold].index
    data_transformed = data_transformed.drop(drop_columns,axis=1)
    #select text columns and convert them to categorical
    text_cols = data_transformed.select_dtypes(include=["object"])
    for col in text_cols:
        data_transformed[col]= data_transformed[col].astype("category")
    # get dummy columns
    dummy_columns = pd.get_dummies(data_transformed.select_dtypes(include=["category"]))
    # append dummy columns
    data_transformed = pd.concat([data_transformed,dummy_columns],axis=1)
    return data_transformed

def train_and_test(data, k=0):
    data_numeric = data.select_dtypes(include=['integer','float'])
    features = data_numeric.columns.drop("SalePrice")
    lr=linear_model.LinearRegression()
    if k == 0:
        train = data[:1460]
        test = data[1460:]
        lr.fit(train[features],train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"],predictions)
        rmse = np.sqrt(mse)
        return rmse 
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = data.sample(frac=1, )
        train = shuffled_df[:1460]
        test = shuffled_df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(data):
            train = data.iloc[train_index]
            test = data.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse


In [34]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)
rmse

[35937.371548352581, 28253.337948087483, 23565.047355142779, 27879.334396318518]


28908.772811975341